# IO

### 流式输出原理

In [1]:
def hello():
    return "Textlong很棒！"

for x in hello():
    print(x, end="|")

T|e|x|t|l|o|n|g|很|棒|！|

In [2]:
def hello():
    yield "Textlong"
    yield "很"
    yield "棒！"

for x in hello():
    print(x, end="|")

Textlong|很|棒！|

In [3]:
def hello():
    def _hello():
        yield "OK: "
        for info in ['Textlong', '很', '棒!']:
            yield info
    return _hello()

for x in hello():
    print(x, end="|")

OK: |Textlong|很|棒!|

In [23]:
def _yield(x):
    yield x

def hello():
    for x in _yield("Textlong"):
        yield x
    for x in _yield("很"):
        yield x
    for x in _yield("棒！"):
        yield x

for x in hello():
    print(x, end="|")

Textlong|很|棒！|

## 流式日志打印

In [2]:
from textlong.io import TextBlock, log

def hello():
    yield TextBlock("info", "评论 >>")
    yield TextBlock("chunk", "Textlong")
    yield TextBlock("chunk", "很")
    yield TextBlock("chunk", "棒！")

log(hello())

[INFO] 评论 >>
Textlong很棒！



'Textlong很棒！'

### 使用队列实现流式消息

In [3]:
qlog = QueueLog()
qlog(hello)

In [4]:
qlog

QueueLog(queue=<multiprocessing.queues.Queue object at 0x10ab52440>)

<div class="alert alert-info">
    ⚠️ 如果之前没有调用，可能会导致一直等待。
</div>

In [5]:
for block in qlog:
    if block.block_type in ['chunk']:
        print(block.text, end="|")
    else:
        print(block.text)

hello textlong
Textlong|很|棒！|

## QueueLog

### achat

In [1]:
from textlong.llm import fake_llm, zhipu, qwen
from textlong.desk import Desk
from textlong.io import QueueLog

qlog = QueueLog()
d = Desk(zhipu, logger=qlog)

In [2]:
d.logger

QueueLog(queue=<multiprocessing.queues.Queue object at 0x10e600400>)

In [3]:
d.achat("给我写两行的儿歌")

for block in qlog:
    if block:
        if block.block_type in ['chunk']:
            print(block.text, end="|")
        else:
            print(block.text)

小|花朵|，|开|在|阳光|里|，
小|蝴蝶|，|飞|舞|笑声|中|。|--end--


### 工具回调

In [4]:
from textlong.llm.tools import tool, convert_to_openai_tool
from textlong.llm import qwen
from textlong.desk import Desk
from textlong.io import QueueLog
import json

@tool
def get_current_weather(location: str):
    """获取城市的天气情况"""
    return f"{location}今天是晴天。 "

qlog = QueueLog()
messages = []
d = Desk(
    qwen,
    toolkits=[get_current_weather],
    tools=[convert_to_openai_tool(get_current_weather)],
    logger=qlog
)

In [5]:
d.achat("根据广州天气，给我写两行的儿歌")

for block in qlog:
    if block:
        if block.block_type in ['chunk']:
            print(block.text, end="|")
        else:
            print(block.text)

{"index": 0, "id": "call_8b6dbfd5115d4b5394fc6f", "type": "function", "function": {"name": "get_current_weather", "arguments": ""}}
{"index": 0, "id": "", "type": "function", "function": {"arguments": "{\"location\": \""}}
{"index": 0, "id": "", "type": "function", "function": {"arguments": "广州\"}"}}
{"index": 0, "id": "", "type": "function", "function": {}}
{"0": {"index": 0, "id": "call_8b6dbfd5115d4b5394fc6f", "type": "function", "function": {"name": "get_current_weather", "arguments": "{\"location\": \"广州\"}"}}}
广州今天是晴天。 
太|阳公公笑眯|眯，广州天空晴万里。
白云|飘飘映蓝幕，出门玩耍|好心情。|--end--|


### 数据分析

In [1]:
import pandas as pd

from textlong.llm import qwen, fake_llm
from textlong.desk import Desk
from textlong.io import QueueLog

qlog = QueueLog()
d = Desk(qwen, logger=qlog)

d.add_dataset(
    "考试成绩",
    pd.DataFrame({
        "姓名": ["薛一凡", "肖一笑", "赖国良", "吴红兵"],
        "语文": [90, 80, 105, 110],
        "数学": [135, 110, 120, 90]
    })
)

d.achat("请查询工具，看看谁的语文成绩最好？")

for block in qlog:
    if block:
        if block.block_type in ['chunk']:
            print(block.text, end="|")
        else:
            print(block.text)

{"index": 0, "id": "call_bb36eb080a20446fbd70f7", "type": "function", "function": {"name": "python_code", "arguments": ""}}
{"index": 0, "id": "", "type": "function", "function": {"arguments": "{\"question\": \"谁"}}
{"index": 0, "id": "", "type": "function", "function": {"arguments": "的语文成绩最好？\"}"}}
{"index": 0, "id": "", "type": "function", "function": {}}
{"index": 0, "id": "", "type": "function", "function": {}}
{"0": {"index": 0, "id": "call_bb36eb080a20446fbd70f7", "type": "function", "function": {"name": "python_code", "arguments": "{\"question\": \"谁的语文成绩最好？\"}"}}}
```|python|
|def main():
    #| 加载考试成绩数据集
   | exam_scores = data['考试成绩'].|df
    
    # 找出语文|成绩最好的学生
    best_chinese|_student = exam_scores.loc[exam_scores|['语文'].idxmax()]['姓名|']
    
    return best_chinese_student|

main()
```||吴红兵
语文|成绩最好的是吴红|兵。|--end--
|

In [4]:
for block in qlog:
    if block.block_type in ['chunk']:
        print(block.text, end="|")
    else:
        print(block.text)

语文|成绩最好的学生是吴|红兵。||

## ZeroMQLog

In [1]:
from textlong.llm import fake_llm, zhipu, qwen
from textlong.desk import Desk
from textlong.io import ZeroMQLog

zlog = ZeroMQLog()
d = Desk(qwen, logger=zlog)

In [2]:
d.achat("给我写两行的儿歌")

for block in zlog:
    if block:
        if block.block_type in ['chunk']:
            print(block.text, end="|")
        else:
            print(block.text)

小|鸭|子|，嘎嘎叫，|
摇摇摆摆水里漂。|||

In [5]:
zlog

ZeroMQLog(socket=<zmq.Socket(zmq.PUSH) at 0x1164a10c0>)

In [6]:
import pandas as pd

from textlong.llm import qwen, fake_llm
from textlong.desk import Desk
from textlong.io import ZeroMQLog

zlog = ZeroMQLog()
d = Desk(qwen, logger=zlog)

d.add_dataset(
    "考试成绩",
    pd.DataFrame({
        "姓名": ["薛一凡", "肖一笑", "赖国良", "吴红兵"],
        "语文": [90, 80, 105, 110],
        "数学": [135, 110, 120, 90]
    })
)

d.achat("请查询工具，看看谁的语文成绩最好？")

for block in zlog:
    if block:
        if block.block_type in ['chunk']:
            print(block.text, end="|")
        else:
            print(block.text)

{"index": 0, "id": "call_b39a516a5cd44e7f8a5783", "type": "function", "function": {"name": "python_code", "arguments": ""}}
{"index": 0, "id": "", "type": "function", "function": {"arguments": "{\"question\": \"谁"}}
{"index": 0, "id": "", "type": "function", "function": {"arguments": "的语文成绩最好？\"}"}}
{"index": 0, "id": "", "type": "function", "function": {}}
{"index": 0, "id": "", "type": "function", "function": {}}
{"0": {"index": 0, "id": "call_b39a516a5cd44e7f8a5783", "type": "function", "function": {"name": "python_code", "arguments": "{\"question\": \"谁的语文成绩最好？\"}"}}}
```|python|
|def main():
    #| 加载考试成绩数据集
   | exam_scores = data["考试成绩"].|df
    
    # 找出语文|成绩最好的学生
    best_chinese|_student = exam_scores.loc[exam_scores|['语文'].idxmax()]['姓名|']
    
    return best_chinese_student|

main()
```||吴红兵
语文|成绩最好的是吴红|兵。||